<img src="https://github.com/pmservice/ai-openscale-tutorials/raw/master/notebooks/images/banner.png" align="left" alt="banner">

# Working with SPSS Collaboration and Deployment Services

This notebook shows how to log the payload for the model deployed on custom model serving engine using Watson OpenScale python sdk.

Contents
- [1. Setup](#setup)
- [2. Binding machine learning engine](#binding)
- [3. Subscriptions](#subscription)
- [4. Scoring and payload logging](#scoring)
- [5. Feedback logging](#feedback)
- [6. Data Mart](#datamart)

<a id="setup"></a>
## 1. Setup

### 1.0 Sample model creation using SPSS Modeler

- Download training data set from [here](https://github.com/pmservice/wml-sample-models/raw/master/spark/product-line-prediction/data/GoSales_Tx.csv)
- Download SPSS Modeler stream from [here](https://gallery.cortanaintelligence.com/Experiment/German-credit-risk-created-on-1-9-2019)
- Deploy the model using SPSS C&DS as web service

### 1.1 Installation and authentication

In [1]:
!pip install --upgrade ibm-ai-openscale --no-cache | tail -n 1

!pip install --upgrade --index-url https://test.pypi.org/simple/ ibm-ai-openscale --no-cache | tail -n 1

Import and initiate.

In [ ]:
from ibm_ai_openscale import APIClient4ICP
from ibm_ai_openscale.supporting_classes import PayloadRecord
from ibm_ai_openscale.supporting_classes.enums import InputDataType, ProblemType
from ibm_ai_openscale.engines import *
from ibm_ai_openscale.utils import *
import urllib3
import jaydebeapi
import os

urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

#### Let's define some constants required to set up data mart:

- AIOS_CREDENTIALS (ICP)
- DATABASE_CREDENTIALS (DB2 on ICP)
- SCHEMA_NAME

In [2]:
AIOS_CREDENTIALS = {
    "url": "***",
    "username": "***",
    "password": "***"
}

In [3]:
# The code was removed by Watson Studio for sharing.

In [4]:
DATABASE_CREDENTIALS = {
    "jdbcurl": "***",
    "hostname": "***",
    "username": "***",
    "password": "***",
    "port": 50000,
    "db": "***",
    "dsn": "***",
    "uri": "***"
}

In [5]:
# The code was removed by Watson Studio for sharing.

In [6]:
SCHEMA_NAME = 'SPSSNOTEBOOK'

In [ ]:
client = APIClient4ICP(AIOS_CREDENTIALS)

In [ ]:
client.version

### 1.2 DataMart setup

In [11]:
client.data_mart.setup(db_credentials=DATABASE_CREDENTIALS, schema=SCHEMA_NAME)

In [44]:
data_mart_details = client.data_mart.get_details()

<a id="binding"></a>
## 2. Bind machine learning engines

### 2.1 Bind  `SPSS C&DS` machine learning engine

Provide credentials using following fields:
- `username`
- `password`
- `url`

In [4]:
SPSS_CDS_ENGINE_CREDENTIALS = {
        "username": "***",
        "password": "***",
        "url": "***",
    }

In [13]:
# The code was removed by Watson Studio for sharing.

In [14]:
binding_uid = client.data_mart.bindings.add('My SPSS C&DS engine', SPSSMachineLearningInstance(SPSS_CDS_ENGINE_CREDENTIALS))

In [45]:
bindings_details = client.data_mart.bindings.get_details()

In [46]:
client.data_mart.bindings.list()

4d880684-e7d1-4d80-aea5-0aa222eb7a03,My SPSS C&DS engine,spss_collaboration_and_deployment_services,2019-01-31T14:27:31.969Z


<a id="subsciption"></a>
## 3. Subscriptions

### 3.1 Add subscriptions

List available deployments.

**Note:** Depending on number of assets it may take some time.

In [47]:
client.data_mart.bindings.list_assets()

091edbc5c4e74be500000167bc5c18e058bf,credit_risk_lr,0000-01-01T00:00:00.0Z,model,,4d880684-e7d1-4d80-aea5-0aa222eb7a03,True
091edbc5c4e74be5000001686a7c7110ac0e,credit_risk_c50_default,0000-01-01T00:00:00.0Z,model,,4d880684-e7d1-4d80-aea5-0aa222eb7a03,False
091edbc5c4e74be5000001686a7c7110a1b1,ai_credit_risk_blogistic,0000-01-01T00:00:00.0Z,model,,4d880684-e7d1-4d80-aea5-0aa222eb7a03,False
091edbc5c4e74be5000001686a7c7110ac36,credit_risk_gle_default,0000-01-01T00:00:00.0Z,model,,4d880684-e7d1-4d80-aea5-0aa222eb7a03,False
091edbc5c4e74be5000001686a7c7110ac97,credit_risk_random_trees_default,0000-01-01T00:00:00.0Z,model,,4d880684-e7d1-4d80-aea5-0aa222eb7a03,False
091edbc5418240f600000166f5190140d404,Drug1n,0000-01-01T00:00:00.0Z,model,,4d880684-e7d1-4d80-aea5-0aa222eb7a03,False
091edbc5c4e74be5000001686a7c7110ac66,credit_risk_lsvm_default,0000-01-01T00:00:00.0Z,model,,4d880684-e7d1-4d80-aea5-0aa222eb7a03,False
091edbc5c4e74be5000001686a7c7110990e,ai_credit_risk_mlogistic,0000-01-01T00:00:00.0Z,model,,4d880684-e7d1-4d80-aea5-0aa222eb7a03,False
091edbc5c4e74be5000001686a7c71109872,ai_credit_risk_btree,0000-01-01T00:00:00.0Z,model,,4d880684-e7d1-4d80-aea5-0aa222eb7a03,False
091edbc5c4e74be5000001686a7c7110ac2e,credit_risk_crt_default,0000-01-01T00:00:00.0Z,model,,4d880684-e7d1-4d80-aea5-0aa222eb7a03,False
091edbc5c4e74be5000001686a7c7110ac7e,credit_risk_quest_all,0000-01-01T00:00:00.0Z,model,,4d880684-e7d1-4d80-aea5-0aa222eb7a03,False


**Action:** Assign your credit risk model source_uid to `source_uid` variable below.

In [18]:
source_uid = '091edbc5c4e74be500000167bc5c18e058bf'

In [19]:
subscription = client.data_mart.subscriptions.add(
    SPSSMachineLearningAsset(source_uid=source_uid,
                             binding_uid=binding_uid,
                             input_data_type=InputDataType.STRUCTURED,
                             problem_type=ProblemType.BINARY_CLASSIFICATION,
                             label_column="Risk",
                             prediction_column="$L-Risk",
                             class_probability_columns=["$LP-Risk", "$LRP-Risk"]))

#### Get subscriptions list

In [48]:
subscriptions = client.data_mart.subscriptions.get_details()

In [49]:
subscriptions_uids = client.data_mart.subscriptions.get_uids()
print(subscriptions_uids)

['091edbc5c4e74be500000167bc5c18e058bf']


#### List subscriptions

In [50]:
client.data_mart.subscriptions.list()

091edbc5c4e74be500000167bc5c18e058bf,credit_risk_lr,model,4d880684-e7d1-4d80-aea5-0aa222eb7a03,2019-01-31T14:27:49.861Z


<a id="scoring"></a>
## 4. Scoring and payload logging

### 4.1 Score the credit risk model and measure response time

In [51]:
import requests
from requests.auth import HTTPBasicAuth
import time
import json

subscription_details = subscription.get_details()
scoring_endpoint = subscription_details['entity']['deployments'][0]['scoring_endpoint']['url']
input_table_id = subscription_details['entity']['asset_properties']['input_data_schema']['id']
id = subscription_details['entity']['asset']['name']

scoring_payload = {'requestInputTable': [{'name': 'credit_risk_training.csv', 'requestInputRow': [{'input': [
            {'name': 'CheckingStatus', 'value': '0_to_200'}, {'name': 'LoanDuration', 'value': '31'},
            {'name': 'CreditHistory', 'value': 'credits_paid_to_date'}, {'name': 'LoanPurpose', 'value': 'other'},
            {'name': 'LoanAmount', 'value': '1889'}, {'name': 'ExistingSavings', 'value': '100_to_500'},
            {'name': 'EmploymentDuration', 'value': 'less_1'}, {'name': 'InstallmentPercent', 'value': '3'},
            {'name': 'Sex', 'value': 'female'}, {'name': 'OthersOnLoan', 'value': 'none'},
            {'name': 'CurrentResidenceDuration', 'value': '3'}, {'name': 'OwnsProperty', 'value': 'savings_insurance'},
            {'name': 'Age', 'value': '32'}, {'name': 'InstallmentPlans', 'value': 'none'},
            {'name': 'Housing', 'value': 'own'}, {'name': 'ExistingCreditsCount', 'value': '1'},
            {'name': 'Job', 'value': 'skilled'}, {'name': 'Dependents', 'value': '1'},
            {'name': 'Telephone', 'value': 'none'}, {'name': 'ForeignWorker', 'value': 'yes'},
            {'name': 'Risk', 'value': ''}]}]}], 'id': id}

start_time = time.time()
resp_score = requests.post(url=scoring_endpoint, json=scoring_payload, auth=HTTPBasicAuth(username=SPSS_CDS_ENGINE_CREDENTIALS['username'], password=SPSS_CDS_ENGINE_CREDENTIALS['password']))

print(resp_score.text)
response_time = int((time.time() - start_time)*1000)
result = resp_score.json()

print(str(result))

{"providedBy":"credit_risk_lr","id":"763bef38-2ed2-4fb1-8d90-b0da6f0f4d4d","columnNames":{"name":["CheckingStatus","LoanDuration","CreditHistory","LoanPurpose","LoanAmount","ExistingSavings","EmploymentDuration","InstallmentPercent","Sex","OthersOnLoan","CurrentResidenceDuration","OwnsProperty","Age","InstallmentPlans","Housing","ExistingCreditsCount","Job","Dependents","Telephone","ForeignWorker","Risk","$L-Risk","$LP-Risk","$LRP-Risk"]},"rowValues":[{"value":[{"value":"0_to_200"},{"value":"31"},{"value":"credits_paid_to_date"},{"value":"other"},{"value":"1889"},{"value":"100_to_500"},{"value":"less_1"},{"value":"3"},{"value":"female"},{"value":"none"},{"value":"3"},{"value":"savings_insurance"},{"value":"32"},{"value":"none"},{"value":"own"},{"value":"1"},{"value":"skilled"},{"value":"1"},{"value":"none"},{"value":"yes"},{"value":""},{"value":"No Risk"},{"value":"0.9042000385999679"},{"value":"0.09579996140003211"}]}],"returnedRequestInputTable":[{"id":"id17STY5BAZ1Y","name":"credit_

### 4.2 Store the request and response in payload logging table

#### Store the payload using Python SDK

**Hint:** You can embed payload logging code into your application so it is logged automatically each time you score the model.

In [52]:
records_list = [PayloadRecord(request=scoring_payload, response=result, response_time=response_time), 
                PayloadRecord(request=scoring_payload, response=result, response_time=response_time)]

for i in range(1, 10):
    records_list.append(PayloadRecord(request=scoring_payload, response=result, response_time=response_time))

subscription.payload_logging.store(records=records_list)

print("Waiting 10 seconds for propagation...")
time.sleep(10)

Waiting 10 seconds for propagation...


<a id="feedback"></a>
## 5. Feedback logging & quality (accuracy) monitoring

### Enable quality monitoring

You need to provide the monitoring `threshold` and `min_records` (minimal number of feedback records).

In [25]:
subscription.quality_monitoring.enable(threshold=0.7, min_records=10)

### Feedback records logging

Feedback records are used to evaluate your model. The predicted values are compared to real values (feedback records).

You can check the schema of feedback table using below method.

In [53]:
subscription.feedback_logging.print_table_schema()

CheckingStatus,string,True
LoanDuration,string,True
CreditHistory,string,True
LoanPurpose,string,True
LoanAmount,string,True
ExistingSavings,string,True
EmploymentDuration,string,True
InstallmentPercent,string,True
Sex,string,True
OthersOnLoan,string,True
CurrentResidenceDuration,string,True


The feedback records can be send to feedback table using below code.

#### Store feedback using CSV format from string

In [54]:
from ibm_ai_openscale.supporting_classes.enums import FeedbackFormat

feedback_records = "0_to_200,31,credits_paid_to_date,other,1889,100_to_500,less_1,3,female,none,3,savings_insurance,32,none,own,1,skilled,1,none,yes,Risk"

for i in range(0, 10):
    feedback_records = feedback_records + '\n' + feedback_records

subscription.feedback_logging.store(feedback_data=feedback_records, feedback_format=FeedbackFormat.CSV)

#### Store feedback using CSV format from file

In [5]:
!wget https://raw.githubusercontent.com/pmservice/wml-sample-models/master/spss/credit-risk/data/credit_risk_feedback.csv

--2019-02-01 10:18:28--  https://raw.githubusercontent.com/pmservice/wml-sample-models/master/spss/credit-risk/data/credit_risk_feedback.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.48.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.48.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 40768 (40K) [text/plain]
Saving to: ‘credit_risk_feedback.csv’

100%[======================================>] 40,768      --.-K/s   in 0.002s  

2019-02-01 10:18:28 (20.4 MB/s) - ‘credit_risk_feedback.csv’ saved [40768/40768]



In [55]:
with open('credit_risk_feedback.csv', 'rb') as csvFile:
    subscription.feedback_logging.store(feedback_data=csvFile, feedback_format=FeedbackFormat.CSV, data_header=True, data_delimiter=',')

In [56]:
subscription.feedback_logging.describe_table()

       LoanDuration    LoanAmount  InstallmentPercent  \
count  13072.000000  13072.000000        13072.000000   
mean      23.650704   3106.216187            2.986536   
std       10.578620   2278.356968            0.985769   
min        4.000000    250.000000            1.000000   
25%       15.000000   1889.000000            2.000000   
50%       26.000000   2073.000000            3.000000   
75%       31.000000   4701.250000            4.000000   
max       64.000000  11676.000000            6.000000   

       CurrentResidenceDuration           Age  ExistingCreditsCount  \
count              13072.000000  13072.000000          13072.000000   
mean                   2.888464     35.008262              1.356334   
std                    0.977733      9.461142              0.532488   
min                    1.000000     19.000000              1.000000   
25%                    2.000000     30.000000              1.000000   
50%                    3.000000     32.000000              1

### Run quality monitoring on demand

By default, quality monitoring is run on hourly schedule. You can also trigger it on demand using below code.

In [57]:
run_details = subscription.quality_monitoring.run(background_mode=False)




 Waiting for end of quality monitoring run 0d1611f3-3853-4b18-ab19-408fc6facf5c 




initializing.
running
completed

---------------------------
 Successfully finished run 
---------------------------




### Show the quality metrics

In [58]:
subscription.quality_monitoring.show_table()

2019-01-31 14:29:00.336000+00:00,0.6820698807446189,0.7,4d880684-e7d1-4d80-aea5-0aa222eb7a03,091edbc5c4e74be500000167bc5c18e058bf,credit_risk_lr,Accuracy_evaluation_3c7dd95d-75a5-4fce-808a-d078a41ef460,
2019-01-31 15:28:26.733000+00:00,0.6820698807446189,0.7,4d880684-e7d1-4d80-aea5-0aa222eb7a03,091edbc5c4e74be500000167bc5c18e058bf,credit_risk_lr,Accuracy_evaluation_873bd20f-459a-4538-b35a-87bcef9e4082,
2019-01-31 15:46:16.213000+00:00,0.6660663007833076,0.7,4d880684-e7d1-4d80-aea5-0aa222eb7a03,091edbc5c4e74be500000167bc5c18e058bf,credit_risk_lr,Accuracy_evaluation_0d1611f3-3853-4b18-ab19-408fc6facf5c,


Get all calculated metrics.

In [59]:
deployment_uids = subscription.get_deployment_uids()

In [60]:
subscription.quality_monitoring.get_metrics(deployment_uid=deployment_uids[0])

{'end': '2019-01-31T15:47:09.924963Z',
 'metrics': [{'process': 'Accuracy_evaluation_3c7dd95d-75a5-4fce-808a-d078a41ef460',
   'timestamp': '2019-01-31T14:29:00.336Z',
   'value': {'metrics': [{'name': 'areaUnderROC', 'value': 0.6820698807446189},
     {'name': 'areaUnderPR', 'value': 0.36673970362466657}],
    'quality': 0.6820698807446189,
    'threshold': 0.7}},
  {'process': 'Accuracy_evaluation_873bd20f-459a-4538-b35a-87bcef9e4082',
   'timestamp': '2019-01-31T15:28:26.733Z',
   'value': {'metrics': [{'name': 'areaUnderROC', 'value': 0.6820698807446189},
     {'name': 'areaUnderPR', 'value': 0.36673970362466657}],
    'quality': 0.6820698807446189,
    'threshold': 0.7}},
  {'process': 'Accuracy_evaluation_0d1611f3-3853-4b18-ab19-408fc6facf5c',
   'timestamp': '2019-01-31T15:46:16.213Z',
   'value': {'metrics': [{'name': 'areaUnderROC', 'value': 0.6660663007833076},
     {'name': 'areaUnderPR', 'value': 0.3365131212972492}],
    'quality': 0.6660663007833076,
    'threshold': 0.7}

<a id="datamart"></a>
## 6. Get the logged data

### 6.1 Payload logging

#### Show (preview) the table

In [38]:
subscription.payload_logging.show_table()

353c1a50-b3fc-44db-a31e-b436b18e2d74-1,2019-01-31 14:28:15.297799+00:00,credit_risk_lr,None,0_to_200,31,credits_paid_to_date,other,1889,100_to_500,less_1,3,female,none,3,savings_insurance,32,none,own,1,skilled,1,none,yes,,No Risk,0.9042000385999679,0.09579996140003211
070157fb-c316-4b80-bd93-6a63fec42ca6-1,2019-01-31 14:28:15.297778+00:00,credit_risk_lr,None,0_to_200,31,credits_paid_to_date,other,1889,100_to_500,less_1,3,female,none,3,savings_insurance,32,none,own,1,skilled,1,none,yes,,No Risk,0.9042000385999679,0.09579996140003211
42e776fd-e4f8-4908-8862-8db26d6cabb7-1,2019-01-31 14:28:15.297756+00:00,credit_risk_lr,None,0_to_200,31,credits_paid_to_date,other,1889,100_to_500,less_1,3,female,none,3,savings_insurance,32,none,own,1,skilled,1,none,yes,,No Risk,0.9042000385999679,0.09579996140003211
86014e49-e941-41a4-b4e0-e91bbb945417-1,2019-01-31 14:28:15.297735+00:00,credit_risk_lr,None,0_to_200,31,credits_paid_to_date,other,1889,100_to_500,less_1,3,female,none,3,savings_insurance,32,none,own,1,skilled,1,none,yes,,No Risk,0.9042000385999679,0.09579996140003211
1c853762-8896-427f-940a-8065e4d11eb9-1,2019-01-31 14:28:15.297714+00:00,credit_risk_lr,None,0_to_200,31,credits_paid_to_date,other,1889,100_to_500,less_1,3,female,none,3,savings_insurance,32,none,own,1,skilled,1,none,yes,,No Risk,0.9042000385999679,0.09579996140003211
1432a3ca-5d46-4063-838e-b7c1e7479ae4-1,2019-01-31 14:28:15.297693+00:00,credit_risk_lr,None,0_to_200,31,credits_paid_to_date,other,1889,100_to_500,less_1,3,female,none,3,savings_insurance,32,none,own,1,skilled,1,none,yes,,No Risk,0.9042000385999679,0.09579996140003211
4a8cdf12-0a8b-473d-bd8c-5be5a6185aed-1,2019-01-31 14:28:15.297672+00:00,credit_risk_lr,None,0_to_200,31,credits_paid_to_date,other,1889,100_to_500,less_1,3,female,none,3,savings_insurance,32,none,own,1,skilled,1,none,yes,,No Risk,0.9042000385999679,0.09579996140003211
bfc81a00-bfbe-468c-b006-4dbf304ecbab-1,2019-01-31 14:28:15.297650+00:00,credit_risk_lr,None,0_to_200,31,credits_paid_to_date,other,1889,100_to_500,less_1,3,female,none,3,savings_insurance,32,none,own,1,skilled,1,none,yes,,No Risk,0.9042000385999679,0.09579996140003211
2b2f8ef9-e2c8-4eb8-a0f1-1ccc3cee8d9a-1,2019-01-31 14:28:15.297626+00:00,credit_risk_lr,None,0_to_200,31,credits_paid_to_date,other,1889,100_to_500,less_1,3,female,none,3,savings_insurance,32,none,own,1,skilled,1,none,yes,,No Risk,0.9042000385999679,0.09579996140003211
1a68b058-be06-40f8-a144-ee1a19c3571a-1,2019-01-31 14:28:15.297597+00:00,credit_risk_lr,None,0_to_200,31,credits_paid_to_date,other,1889,100_to_500,less_1,3,female,none,3,savings_insurance,32,none,own,1,skilled,1,none,yes,,No Risk,0.9042000385999679,0.09579996140003211


#### Return the table content as pandas dataframe

In [99]:
pandas_df = subscription.payload_logging.get_table_content(format='pandas')

### 6.2 Feedback logging

Check the schema of table.

In [100]:
subscription.feedback_logging.print_table_schema()

CheckingStatus,string,True
LoanDuration,string,True
CreditHistory,string,True
LoanPurpose,string,True
LoanAmount,string,True
ExistingSavings,string,True
EmploymentDuration,string,True
InstallmentPercent,string,True
Sex,string,True
OthersOnLoan,string,True
CurrentResidenceDuration,string,True


Preview table content.

In [101]:
subscription.feedback_logging.show_table()

0_to_200,31,credits_paid_to_date,other,1889,100_to_500,less_1,3,female,none,3,savings_insurance,32,none,own,1,skilled,1,none,yes,Risk,2019-01-30 07:47:16.470000+00:00
0_to_200,31,credits_paid_to_date,other,1889,100_to_500,less_1,3,female,none,3,savings_insurance,32,none,own,1,skilled,1,none,yes,Risk,2019-01-30 07:47:16.470000+00:00
0_to_200,31,credits_paid_to_date,other,1889,100_to_500,less_1,3,female,none,3,savings_insurance,32,none,own,1,skilled,1,none,yes,Risk,2019-01-30 07:47:16.470000+00:00
0_to_200,31,credits_paid_to_date,other,1889,100_to_500,less_1,3,female,none,3,savings_insurance,32,none,own,1,skilled,1,none,yes,Risk,2019-01-30 07:47:16.470000+00:00
0_to_200,31,credits_paid_to_date,other,1889,100_to_500,less_1,3,female,none,3,savings_insurance,32,none,own,1,skilled,1,none,yes,Risk,2019-01-30 07:47:16.470000+00:00
0_to_200,31,credits_paid_to_date,other,1889,100_to_500,less_1,3,female,none,3,savings_insurance,32,none,own,1,skilled,1,none,yes,Risk,2019-01-30 07:47:16.470000+00:00
0_to_200,31,credits_paid_to_date,other,1889,100_to_500,less_1,3,female,none,3,savings_insurance,32,none,own,1,skilled,1,none,yes,Risk,2019-01-30 07:47:16.470000+00:00
0_to_200,31,credits_paid_to_date,other,1889,100_to_500,less_1,3,female,none,3,savings_insurance,32,none,own,1,skilled,1,none,yes,Risk,2019-01-30 07:47:16.470000+00:00
0_to_200,31,credits_paid_to_date,other,1889,100_to_500,less_1,3,female,none,3,savings_insurance,32,none,own,1,skilled,1,none,yes,Risk,2019-01-30 07:47:16.470000+00:00
0_to_200,31,credits_paid_to_date,other,1889,100_to_500,less_1,3,female,none,3,savings_insurance,32,none,own,1,skilled,1,none,yes,Risk,2019-01-30 07:47:16.470000+00:00


Describe table (calulcate basic statistics).

In [55]:
subscription.feedback_logging.describe_table()

       LoanAmount  LoanDuration  InstallmentPercent  ExistingCreditsCount  \
count        20.0          20.0                20.0                  20.0   
mean        884.0          18.0                 4.0                   1.0   
std           0.0           0.0                 0.0                   0.0   
min         884.0          18.0                 4.0                   1.0   
25%         884.0          18.0                 4.0                   1.0   
50%         884.0          18.0                 4.0                   1.0   
75%         884.0          18.0                 4.0                   1.0   
max         884.0          18.0                 4.0                   1.0   

        Age  Dependents  CurrentResidenceDuration  
count  20.0        20.0                      20.0  
mean   36.0         2.0                       4.0  
std     0.0         0.0                       0.0  
min    36.0         2.0                       4.0  
25%    36.0         2.0                       

Get table content.

In [102]:
feedback_pd = subscription.feedback_logging.get_table_content(format='pandas')

### 6.3 Quality metrics table

In [103]:
subscription.quality_monitoring.print_table_schema()

ts,timestamp,False
quality,float,False
quality_threshold,float,False
binding_id,string,False
subscription_id,string,False
deployment_id,string,True
process,string,False
asset_revision,string,True


In [104]:
subscription.quality_monitoring.show_table()

2019-01-30 15:47:28.347000+00:00,1.0,0.7,e30a1703-a74c-4066-8994-566c24857bae,091edbc5c4e74be500000167bc5c18e058bf,credit_risk_lr,Accuracy_evaluation_3357659d-a791-4696-af3e-10e98612e984,


### 6.4 Performance metrics table

In [105]:
subscription.performance_monitoring.print_table_schema()

ts,timestamp,False
scoring_time,float,False
scoring_records,object,False
binding_id,string,False
subscription_id,string,False
deployment_id,string,True
process,string,False
asset_revision,string,True


In [70]:
subscription.performance_monitoring.show_table()

2019-01-31 11:02:16.533079+00:00,369.0,1,7f9eeb14-3e89-4fd6-a86a-eaab255f02e4,091edbc5c4e74be500000167bc5c18e058bf,credit_risk_lr,,
2019-01-31 11:02:16.533164+00:00,369.0,1,7f9eeb14-3e89-4fd6-a86a-eaab255f02e4,091edbc5c4e74be500000167bc5c18e058bf,credit_risk_lr,,
2019-01-31 11:02:16.533208+00:00,369.0,1,7f9eeb14-3e89-4fd6-a86a-eaab255f02e4,091edbc5c4e74be500000167bc5c18e058bf,credit_risk_lr,,
2019-01-31 11:02:16.533243+00:00,369.0,1,7f9eeb14-3e89-4fd6-a86a-eaab255f02e4,091edbc5c4e74be500000167bc5c18e058bf,credit_risk_lr,,
2019-01-31 11:02:16.533276+00:00,369.0,1,7f9eeb14-3e89-4fd6-a86a-eaab255f02e4,091edbc5c4e74be500000167bc5c18e058bf,credit_risk_lr,,
2019-01-31 11:02:16.533309+00:00,369.0,1,7f9eeb14-3e89-4fd6-a86a-eaab255f02e4,091edbc5c4e74be500000167bc5c18e058bf,credit_risk_lr,,
2019-01-31 11:02:16.533340+00:00,369.0,1,7f9eeb14-3e89-4fd6-a86a-eaab255f02e4,091edbc5c4e74be500000167bc5c18e058bf,credit_risk_lr,,
2019-01-31 11:02:16.533372+00:00,369.0,1,7f9eeb14-3e89-4fd6-a86a-eaab255f02e4,091edbc5c4e74be500000167bc5c18e058bf,credit_risk_lr,,
2019-01-31 11:02:16.533406+00:00,369.0,1,7f9eeb14-3e89-4fd6-a86a-eaab255f02e4,091edbc5c4e74be500000167bc5c18e058bf,credit_risk_lr,,
2019-01-31 11:02:16.533440+00:00,369.0,1,7f9eeb14-3e89-4fd6-a86a-eaab255f02e4,091edbc5c4e74be500000167bc5c18e058bf,credit_risk_lr,,


### 6.5 Data Mart measurement facts table

In [61]:
data_mart_metrics = client.data_mart.get_deployment_metrics()

---

### Authors
Lukasz Cmielowski, PhD, is an Automation Architect and Data Scientist at IBM with a track record of developing enterprise-level applications that substantially increases clients' ability to turn data into actionable knowledge.